In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.indeed.com/jobs?q=title%3Adata%20%2460%2C000&jt=fulltime&explvl=entry_level&sort=date&limit=50&education=bachelor_degree&vjk=8b05a317f91cca42'

In [3]:
# create list variables to scrape
job_title = []
company_name = []
job_location = []
job_salary = []
detailed_link = []
posted_date = []

In [4]:
# SOME FUNCTIONS

# Scrap web data as text
def get_data(url):
    r = requests.get(url)
    
    return r.text

# Parse all html code
def parse_html(url):
    raw = get_data(url)
    soup = BeautifulSoup(raw, 'lxml')
    
    return soup

def get_job_info(soup):
    # access tag/class with data
    data_str = soup.find_all('div',class_='job_seen_beacon')
    
    for jobs in data_str:
        # job title
        try:
            job = jobs.find('h2')
            title = job.select('span')[1].text
            job_title.append(title)
        except:
            job_title.append(' ')
        
        # company name
        try:
            company = jobs.find('span', class_='companyName').text
            company_name.append(company)
        except:
            company_name.append(' ')
        
        # job location
        try:
            location = jobs.find('div', class_='companyLocation').text
            job_location.append(location)
        except:
            job_location.append(' ')
        
        # salary
        try:
            salary = jobs.find('div', class_='attribute_snippet').text
            job_salary.append(salary)
        except:
            job_salary.append(' ')
        
        # posted date
        try:
            date = jobs.find(class_='date').text.replace('Posted', '')
            posted_date.append(date)
        except:
            company_name.append(' ')
            
    # detailed link
    job_data = soup.find('div', id='mosaic-provider-jobcards')
    for a in job_data.find_all('a', class_='tapItem'):
        try:
            detailed_link.append('indeed.com'+a['href'])  
        except:
            detailed_link.append(' ')
    


In [5]:
# Let the webscraping begin!
soup = parse_html(url)
get_job_info(soup)

In [6]:
# stores as dataframe
scraped = pd.DataFrame({'title': job_title,
                        'company' : company_name,
                        'location' : job_location,
                        'salary' : job_salary,
                        'link' : detailed_link,
                        'date' : posted_date})

In [7]:
# export as csv
scraped.to_csv('jobs.csv', mode='a')